# Creating a Retinanet model

We first install the Retinanet library

In [0]:
!git clone https://github.com/fizyr/keras-retinanet

In [0]:
cd keras-retinanet

In [0]:
!pip install keras==2.3.0

In [0]:
!pip install . --user
!python setup.py build_ext --inplace

In [0]:
import os
os.kill(os.getpid(), 9)

Now, we download the necessary files. The following files are employed for fine-tuning the network and configuring the dataset.

In [0]:
!wget https://www.dropbox.com/s/acio8y3sen0vhqk/build_dataset.py?dl=1 -O build_dataset.py
!wget https://www.dropbox.com/s/roqdmqhgljxaaqe/resnet50_coco_best_v2.1.0.h5?dl=1 -O resnet50_coco_best_v2.1.0.h5

We download the dataset. If you are working with Google Colab, you have several options to download the dataset in this notebook, see the available options in the [LabelDetection documentation](https://github.com/ancasag/LabelDetection).

In [0]:

!unzip dataset.zip

We configure the dataset

In [0]:
!python build_dataset.py

We start the training process.

In [0]:
!python keras-retinanet/keras_retinanet/bin/train.py --batch-size 2 --steps numImgAnot --epochs 20 --weights resnet50_coco_best_v2.1.0.h5 --snapshot-path snapshots csv dataset/retinanet_train.csv dataset/retinanet_classes.csv

Now, we convert the trained model to use it for making predictions.

In [0]:
!python keras-retinanet/keras_retinanet/bin/convert_model.py snapshots/resnet50_csv_20.h5 output.h5

Finally, we evaluate the model to know its performance in an independent test set. 

In [0]:
!python keras-retinanet/keras_retinanet/bin/evaluate.py csv dataset/retinanet_test.csv dataset/classes.csv output.h5

In [0]:
!rm -rf snapshots/resnet50_csv_0*
!rm -rf snapshots/resnet50_csv_1*

At the end you will have your model in the file output.h5. Those weights can be included in the application to be employed with new images. 

------------------------------------


## Data distillation

After training a model with the annotated images, it is possible to apply a data distillation procedure to create a model using the unlabelled images. You can only apply this techique if there were unlabelled images in your dataset. 


In [0]:
!git clone https://github.com/ancasag/ensembleObjectDetection.git

In [0]:
cd /content/ensembleObjectDetection/TestTimeAugmentation

In [0]:
!pip install clodsa
!pip install gluoncv
!pip install mxnet
!pip install mrcnn

In [0]:
import os
os.kill(os.getpid(), 9)

In [ ]:
cd /content/ensembleObjectDetection/TestTimeAugmentation

In [0]:
import testTimeAugmentation
import function
import os
import shutil
import argparse
import ensembleOptions
from mainTTA import tta
from imutils import paths

In [0]:
pathImg = '/content/dataset/unlabelled/'

In [0]:
retinaResnet50 = testTimeAugmentation.RetinaNetResnet50Pred('/content/output.h5', '/content/dataset/classes.csv')

In [0]:
myTechniques = [ "histo","hflip","none"]

In [0]:
option = "consensus"


In [0]:
tta(retinaResnet50,myTechniques,pathImg,option)

In [0]:
cd /content

In [0]:
!ls /content/dataset/unlabelled/*.jpg >> /content/dataset/train.txt

In [0]:
!sed -i 's#/content/dataset/unlabelled/##g' /content/dataset/train.txt

In [0]:
!sed -i 's#.jpg##g' /content/dataset/train.txt

In [0]:
!mv /content/dataset/unlabelled/*.jpg /content/dataset/images/

In [0]:
!mv /content/dataset/unlabelled/*.xml /content/dataset/annotations/

In [0]:
!python build_dataset.py

In [0]:
!python keras-retinanet/keras_retinanet/bin/train.py --batch-size 2 --steps numImg --epochs 20 --weights snapshots/resnet50_csv_20.h5 --snapshot-path snapshotsDD csv dataset/retinanet_train.csv dataset/retinanet_classes.csv

In [0]:
!python keras-retinanet/keras_retinanet/bin/convert_model.py snapshotsDD/resnet50_csv_20.h5 outputDD.h5

In [0]:
!python keras-retinanet/keras_retinanet/bin/evaluate.py csv dataset/retinanet_test.csv dataset/classes.csv outputDD.h5

# Using the model in LabelDetection

If you want to use the trained model with LabelDetection, you must download the following files:
- dataset/classes.csv
- outputDD.h5